Imports

In [1]:
import argparse
from functools import partial
from inspect import currentframe, getframeinfo
from glob import glob
from pathlib import Path
from os.path import join
import os
import pandas as pd
import pickle
import sys

from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
from nipype.interfaces.base import Bunch
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node

from utils.event_utils import process_confounds

180828-02:55:24,381 duecredit ERROR:
	 Failed to import duecredit due to No module named 'duecredit'


### Parse Arguments
These are not needed for the jupyter notebook, but are used after conversion to a script for production

- conversion command:
  - jupyter nbconvert --to script --execute 1stlevel_analysis.ipynb

In [2]:
parser = argparse.ArgumentParser(description='Example BIDS App entrypoint script.')
parser.add_argument('-data_dir', default='/data')
parser.add_argument('-derivatives_dir', default=None)
parser.add_argument('-working_dir', default=None)
parser.add_argument('--subjids', nargs="+")
parser.add_argument('--sessions', nargs="+", default=[1])
parser.add_argument('--runs', nargs="+", default=[1,2])
parser.add_argument('--n_procs', default=16, type=int)
if '-derivatives_dir' in sys.argv or '-h' in sys.argv:
    args = parser.parse_args()
else:
    args = parser.parse_args([])
    args.data_dir = '/data'
    args.derivatives_dir = '/data/derivatives'
    args.subjids = ['s0324']
    args.n_procs=4
    owd = os.getcwd()

### Initial Setup

In [3]:
# get current directory to pass to function nodes
filename = getframeinfo(currentframe()).filename
current_directory = str(Path(filename).resolve().parent)

#### Experiment Variables
derivatives_dir = args.derivatives_dir
fmriprep_dir = join(derivatives_dir, 'fmriprep', 'fmriprep')
data_dir = args.data_dir
first_level_dir = join(derivatives_dir,'1stlevel')
if args.working_dir is None:
    working_dir = join(derivatives_dir, '1stlevel_workingdir')
else:
    working_dir = join(args.working_dir, '1stlevel_workingdir')
n_procs = args.n_procs
# TR of functional images
TR = .71

In [4]:
# print
print('*'*79)
print('Subject(s): %s\n, derivatives_dir: %s\n, data_dir: %s' % 
     (args.subjids, derivatives_dir, data_dir))
print('*'*79)

*******************************************************************************
Subject(s): ['s0324']
, derivatives_dir: /data/derivatives
, data_dir: /data
*******************************************************************************


In [5]:
def get_events_regressors(subject_id, session, run, base_dir):
    from glob import glob
    import pandas as pd
    # strip "sub" from beginning of subject_id if provided
    subject_id = subject_id.replace('sub-','')
    ## Get the Confounds File (output of fmriprep)
    # Read the TSV file and convert to pandas dataframe
    confounds_file = glob(join(fmriprep_dir,
                               'sub-%s' % subject_id,
                               'ses-%s' % session, 'func',
                               '*run-%s*confounds.tsv' % run))
    ## Get the Events File if it exists
    # Read the TSV file and convert to pandas dataframe
    event_file = glob(join(data_dir, 'fmriprep', 'fmriprep',
                           'sub-%s' % subject_id,
                           'ses-%s' % session, 'func',
                           '*run-%s*events.tsv' % run))  
    assert len(event_file) == 1 and len(confounds_file) == 1
    # set up events file
    event_file = event_file[0]
    events_df = pd.read_csv(event_file,sep = '\t')
    regressors, regressor_names = process_confounds(confounds_file[0])
    return events_df, regressors, regressor_names

def get_subject_info(events_df, regressors, regressor_names): 
    # subset events_df
    events_subset = events_df.query('type=="response"')
    EV_dict = {'conditions': ['motor'],
               'onsets': [list(events_subset.onset)],
               'durations': [list(events_subset.duration)],
               'amplitudes': [[1]]}
    contrasts = [['motor', 'T', ['motor'], [1]]]
    subject_info = Bunch(conditions=EV_dict['conditions'],
                        onsets=EV_dict['onsets'],
                        durations=EV_dict['durations'],
                        amplitudes=EV_dict['amplitudes'],
                        tmod=None,
                        pmod=None,
                        regressor_names=regressor_names,
                        regressors=regressors.T.tolist())
    return subject_info, contrasts


# Create the function Nodes
getevents = Node(Function(input_names=['subject_id', 'session', 'run', 'data_dir'],
                          output_names=['events_df', 'regressors', 'regressor_names'],
                         function=get_events_regressors),
                    name='getevents')

getsubjectinfo = Node(Function(input_names=['events_df', 'regressors', 'regressor_names'],
                             output_names=['subject_info', 'contrasts'],
                         function=get_subject_info),
                    name='getsubjectinfo')

### Specify Input and Output Stream

In [9]:
infosource = Node(IdentityInterface(fields=['subject_id',
                                            'session',
                                            'run',
                                           'data_dir']),
                  name="infosource")
infosource.inputs.data_dir = data_dir
infosource.iterables = [('subject_id', args.subjids),
                        ('session', args.sessions),
                        ('run', args.runs)]

# SelectFiles - to grab the data (alternative to DataGrabber)
templates = {'func': join('sub-{subject_id}','ses-{session}','func',
                         '*run-{run}*T1w_preproc.nii.gz'),
             'mask': join('sub-{subject_id}','ses-{session}','func',
                          '*run-{run}*T1w_brainmask.nii.gz')}
selectfiles = Node(SelectFiles(templates,
                               base_directory=fmriprep_dir,
                               sort_filelist=True),
                   name='selectFiles')
    
masker = Node(fsl.maths.ApplyMask(), name='masker')

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=first_level_dir), name="datasink")
# Use the following DataSink output substitutions
substitutions = [('_subject_id_', ''),
                ('fstat', 'FSTST'),
                ('run0.mat', 'designfile.mat')]

datasink.inputs.substitutions = substitutions
"""
# ridiculous regexp substitution to get files just right
# link to ridiculousness: https://regex101.com/r/ljS5zK/3
match_str = "(?P<sub>s[0-9]+)\/(?P<task>[A-Za-z1-9_]+)_(?P<model>model-[a-z]+)_(?P<submodel>wf-[a-z]+)\/(s[0-9]+/|)"
replace_str = "\g<sub>/\g<task>/\g<model>/\g<submodel>/"
regexp_substitutions = [(match_str, replace_str)]
datasink.inputs.regexp_substitutions = regexp_substitutions
"""

'\n# ridiculous regexp substitution to get files just right\n# link to ridiculousness: https://regex101.com/r/ljS5zK/3\nmatch_str = "(?P<sub>s[0-9]+)\\/(?P<task>[A-Za-z1-9_]+)_(?P<model>model-[a-z]+)_(?P<submodel>wf-[a-z]+)\\/(s[0-9]+/|)"\nreplace_str = "\\g<sub>/\\g<task>/\\g<model>/\\g<submodel>/"\nregexp_substitutions = [(match_str, replace_str)]\ndatasink.inputs.regexp_substitutions = regexp_substitutions\n'

In [10]:
# SpecifyModel - Generates FSL-specific Model
modelspec = Node(SpecifyModel(input_units='secs',
                              time_repetition=TR,
                              high_pass_filter_cutoff=80),
                 name="modelspec")
# Level1Design - Creates FSL config file 
level1design = Node(fsl.Level1Design(bases={'dgamma':{'derivs': True}},
                                     interscan_interval=TR,
                                     model_serial_correlations=True),
                        name="level1design")
# FEATmodel generates an FSL design matrix
level1model = Node(fsl.FEATModel(), name="FEATModel")

# FILMGLs
# smooth_autocorr, check default, use FSL default
filmgls = Node(fsl.FILMGLS(threshold=1000), name="GLS")

# Create workflow

In [11]:
"""
select_out = selectfiles.run()

masker.inputs.in_file = select_out.outputs.func
masker.inputs.mask_file = select_out.outputs.mask
mask_out = masker.run()

modelspec.inputs.functional_runs = mask_out.outputs.out_file
modelspec_out = modelspec.run()

level1design.inputs.session_info = modelspec_out.outputs.session_info
level1design_out = level1design.run()

level1model.inputs.ev_files = level1design_out.outputs.ev_files
level1model.inputs.fsf_file = level1design_out.outputs.fsf_files
level1model_out = level1model.run()

filmgls.inputs.design_file = level1model_out.outputs.design_file
filmgls.inputs.tcon_file = level1model_out.outputs.con_file
filmgls.inputs.fcon_file = level1model_out.outputs.fcon_file
filmgls.inputs.in_file = mask_out.outputs.out_file
film_out = filmgls.run()
"""

'\nselect_out = selectfiles.run()\n\nmasker.inputs.in_file = select_out.outputs.func\nmasker.inputs.mask_file = select_out.outputs.mask\nmask_out = masker.run()\n\nmodelspec.inputs.functional_runs = mask_out.outputs.out_file\nmodelspec_out = modelspec.run()\n\nlevel1design.inputs.session_info = modelspec_out.outputs.session_info\nlevel1design_out = level1design.run()\n\nlevel1model.inputs.ev_files = level1design_out.outputs.ev_files\nlevel1model.inputs.fsf_file = level1design_out.outputs.fsf_files\nlevel1model_out = level1model.run()\n\nfilmgls.inputs.design_file = level1model_out.outputs.design_file\nfilmgls.inputs.tcon_file = level1model_out.outputs.con_file\nfilmgls.inputs.fcon_file = level1model_out.outputs.fcon_file\nfilmgls.inputs.in_file = mask_out.outputs.out_file\nfilm_out = filmgls.run()\n'

In [ ]:
#desmtx=np.loadtxt(level1model_out.outputs.design_file,skiprows=5)

In [13]:
# Initiation of the 1st-level analysis workflow
l1analysis = Workflow(name='l1analysis')
l1analysis.base_dir = working_dir
l1analysis.connect([(infosource, selectfiles, [('subject_id', 'subject_id'),
                                               ('session', 'session'),
                                               ('run', 'run')]),
                    (infosource, getevents, [('subject_id', 'subject_id'),
                                             ('session', 'session'),
                                             ('run', 'run')]),
                    (infosource, getevents, [('data_dir', 'data_dir')]),
                    (getevents, getsubjectinfo, [('events_df', 'events_df'),
                              ('regressors', 'regressors'), 
                              ('subjectinfo', 'subjectinfo')]),
                    (selectfiles, masker, [('func','in_file'), ('mask', 'mask_file')]),
                    (masker, modelspec, [('out_file', 'functional_runs')]),
                    (getsubjectinfo, modelspec, [('subject_info', 'subject_info')]),
                    (modelspec, level1design, [('session_info','session_info')]),
                    (getsubjectinfo, level1design, [('contrasts', 'contrasts')]),
                    (level1design, level1model, [('ev_files', 'ev_files'),
                                             ('fsf_files','fsf_file')]),
                    (infosource, datasink, [('subject_id', 'container')]),
                    (level1model, datasink, [('design_file', 'FEAT.@design_file')]),
                    (masker, filmgls, [('out_file', 'in_file')]),
                    (level1model, filmgls, [('design_file', 'design_file'),
                                        ('con_file', 'tcon_file'),
                                        ('fcon_file', 'fcon_file')]),
                    (filmgls, datasink, [('copes', 'filmgls.@copes'),
                                     ('zstats', 'filmgls.@Z'),
                                     ('fstats', 'filmgls.@F'),
                                     ('tstats','filmgls.@T'),
                                     ('param_estimates','filmgls.@param_estimates'),
                                     ('residual4d', 'filmgls.@residual4d'),
                                     ('sigmasquareds', 'filmgls.@sigmasquareds')])])

In [14]:
#l1analysis.run()
l1analysis.run('MultiProc', plugin_args={'n_procs': n_procs})

180828-02:56:47,214 workflow INFO:
	 Workflow l1analysis settings: ['check', 'execution', 'logging', 'monitoring']
180828-02:56:47,443 workflow INFO:
	 Running in parallel.
180828-02:56:47,447 workflow INFO:
	 Currently running 0 tasks, and 4 jobs ready. Free memory (GB): 10.45/10.45, Free processors: 4/4
180828-02:56:47,464 workflow INFO:
	 Executing node l1analysis.selectFiles in dir: /data/derivatives/1stlevel_workingdir/l1analysis/_run_2_session_1_subject_id_s0324/selectFiles
180828-02:56:47,482 workflow INFO:
	 Executing node l1analysis.getevents in dir: /data/derivatives/1stlevel_workingdir/l1analysis/_run_2_session_1_subject_id_s0324/getevents
180828-02:56:47,501 workflow INFO:
	 Executing node l1analysis.selectFiles in dir: /data/derivatives/1stlevel_workingdir/l1analysis/_run_1_session_1_subject_id_s0324/selectFiles
180828-02:56:47,524 workflow INFO:
	 Executing node l1analysis.getevents in dir: /data/derivatives/1stlevel_workingdir/l1analysis/_run_1_session_1_subject_id_s0324

RuntimeError: Workflow did not execute cleanly. Check log for details